In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX",
    "BioSequences",
    "Kmers",
    "Graphs",
    "MetaGraphs",
    "SparseArrays",
    "ProgressMeter",
    "Distributions",
    "HiddenMarkovModels",
    "BioAlignments",
    "StatsBase"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
reference_fastas = sort(filter(x -> occursin(r"\.fna$", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
reference_fasta = first(reference_fastas)

In [ ]:
filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true))

In [ ]:
fastq = first(filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin("filtlong", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true)))

In [ ]:
initial_k = Mycelia.assess_dnamer_saturation([fastq], plot=false)

In [ ]:
initial_kmer_type = Kmers.DNAKmer{initial_k}

In [ ]:
canonical_kmers = collect(keys(Mycelia.count_canonical_kmers(Kmers.DNAKmer{initial_k}, fastq)))

In [ ]:
stranded_kmer_graph = Mycelia.build_stranded_kmer_graph(canonical_kmers, Mycelia.open_fastx(fastq))

In [ ]:
Mycelia.viterbi_maximum_likelihood_traversals(stranded_kmer_graph)